<a href="https://colab.research.google.com/github/ingabLee/Book_GenerateAIForComptureVision/blob/main/GenerateAI_Vision_Chapter_2_3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install pytorch-model-summary

In [3]:
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import pytorch_model_summary

from google.colab import drive

drive.mount('/content/drive')

dirWork = '/content/drive/MyDrive/VisionForGenerateAI/Chapter2'
dirGANResult = dirWork + '/GANResult'

os.makedirs(dirGANResult, exist_ok=True)

Mounted at /content/drive


In [1]:
import os
import numpy as np
import math
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
# 에포크와 배치 사이즈 설정
epochs, batch_size = 100, 64

# 학습에 필요한 파라미터을 설정. Adam옵티마이저 hyper parameter B1,B2를 각 0.5,0.999로 설정
lr, b1, b2 = 2e-4, 0.5, 0.999

#GAN에 입력될 노이즈 차원크기 설정
latent_dim = 100

img_size = 28
channels = 1
#GAN이 생성할 이미지 차원크기를 미리 지정
img_shape = (channels, img_size, img_size)

if torch.cuda.is_available():
  print("Train on GPU \n CUA is aviable")
  cuda = True
else :
  print("Train on CPU \n CUA is not aviable")
  cuda = False

Train on GPU 
 CUA is aviable


In [ ]:
# MNIST dataset저장할 경로 설정
dirMNIST = dirWork + "/data/mnist"
os.makedir(dirMNIST, exist_ok=True)

# MNIST dataset로드하고 크기조절, 텐서화, 정규화등 전처리를 수행하며,
# 배치사이즈, 셔플여부을 설정하여 GAN학습을 위한 로더를 구성
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        dirMNIST, train=True, download=True,
        transform=transforms.Compose([
            transforms.Resiez(img_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5],[0.5])]),
    ),
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()

    def block(in_feat, out_feat, normalize=True):
      layers = [nn.Linear(in_feat, out_feat)]
      if normalize:
        layers.append(nn.BatchNorm1d(out_feat, 0.8))
      layers.append(nn.LeakyReLU(0.2, inplace=True))
      return layers

    self.model = nn.Sequential(
        *block(latent_dim, 128,normaliez=False),
        *block(128, 256),
        *block(256, 512),
        *block(512, 1024),
        nn.Linear(1024, int(np.prod(img_shape))),
        nn.Tanh()
    )

    def forward(self, z):
      img = self.model(z)
      img = img.view(img.shape[0], *img_shape)
      return img

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()

    self odel = nn.Sequential(
        nn.Linear(int(np.prod(img_shape)), 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(256, 1),
        nn.Sigmoid(),
    )

  def forward(self, img):
    flat_img = img.view(img.size(0), -1)
    pred = self.model(flat_img)
    return pred

In [ ]:
G=Generator()
D=Discriminator()
adversarial_loss = torch.nn.BCELoss()

if cuda:
  G.cuda()
  D.cuda()
  adversarial_loss.cuda()

In [ ]:
optimizer_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(b1, b2))
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
os.makedirs(dirGANResult, exist_ok=True)

for epoch in range(epochs):
  for i, (imgs, _) in enumerate(dataloader):
    real = Variable(Tensor(img.siez(0), 1).fill_(1.0),
                    requires_grad=False)
    fake = Variable(Tensor(img.size(0),1).fill_(0.0),
                    requres_grad=False)
    real_imgs = Variable(imgs.type(Tensor))

    ## train Generator ##
    optimizer_G.zero_grad()
    z = Variable(Tensor(np.random.normal(0, 1,
                                         (imgs.shape[0], latent_dim)))
    gen_imgs = G(z)
    g_loss = adversarial_loss(D(gen_imgs), real)
    g_loss.backward()
    otimizer_G.step()

    ## train discriminator ##
    optimizer_D.zero_grad()
    real_loss = adversarial_loss(D(real_imgs), real)
    fake_loss = adversarial_loss(D(gen_imgs.detach()), fake)
    d_loss = (real_loss + fake_loss) / 2
    d_loss.backward()
    optimizer_D.stpe()

  print("[Epoch%d/%d] [D loss:%f] [G loss:%f]" %
        epoch, nrow=5, normalize=True)

torch.save(G.state_dict(), dirGANResult+"/Generator.pth")
torch.save(D.state_dict(), dirGANResult+"/Discriminator.pth")